# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/video_games_sales.csv).

In [1]:
import pandas as pd
import re
from scipy import stats as st
import glob
import nltk
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_csv('video_games_sales.csv')
df.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55792 entries, 0 to 55791
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Rank            55792 non-null  int64  
 1   Name            55792 non-null  object 
 2   basename        55792 non-null  object 
 3   Genre           55792 non-null  object 
 4   ESRB_Rating     23623 non-null  object 
 5   Platform        55792 non-null  object 
 6   Publisher       55792 non-null  object 
 7   Developer       55775 non-null  object 
 8   VGChartz_Score  0 non-null      float64
 9   Critic_Score    6536 non-null   float64
 10  User_Score      335 non-null    float64
 11  Total_Shipped   1827 non-null   float64
 12  Global_Sales    19415 non-null  float64
 13  NA_Sales        12964 non-null  float64
 14  PAL_Sales       13189 non-null  float64
 15  JP_Sales        7043 non-null   float64
 16  Other_Sales     15522 non-null  float64
 17  Year            54813 non-null 

#### Ответьте на следующие вопросы:

Для каждого вопроса:

- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.


**1) Как критики относятся к спортивным играм?**

In [4]:
# выберем данные с отметкой критиков
df_crit = df[['Genre', 'Platform', 'Critic_Score']].dropna()
df_crit

,Genre,Platform,Critic_Score
0,Sports,Wii,7.7
1,Platform,NES,10.0
2,Racing,Wii,8.2
4,Sports,Wii,8.0
5,Role-Playing,GB,9.4
...,...,...,...
55387,Adventure,3DS,5.4
55423,Music,NS,9.0
55490,Action,NS,8.0
55528,Action-Adventure,NS,7.0


**H0: критикам нравятся спортивные игры и средняя оценка >= 7**   
**H1: средняя оценка критиков менее 7**   
уровень значимости определим 0.05

Сравниваем значение по выборке с константой, поэтому нужен одновыборочный критерий.

Тест нужен односторонний, т.к. нас интересует только неравенство в меньшую сторону.

In [5]:
alpha = 0.05
result1 = st.ttest_1samp(df_crit.loc[df_crit['Genre'] == 'Sports'].Critic_Score, 7)
print(result1)

if (result1.statistic < 0) & (result1.pvalue / 2 < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка критиков меньше 7')
else:
    print('Не отвергаем нулевую нулевую гипотезу, критикам нравятся спортивные игры')

Ttest_1sampResult(statistic=5.197156914436623, pvalue=2.671410902970997e-07)
Не отвергаем нулевую нулевую гипотезу, критикам нравятся спортивные игры


**2) Критикам нравятся больше игры на PC или на PS4?**

**H0: средняя оценка для игр на  PC или на PS4 одинаковая**     
**H1: средняя оценка для игр на  PC или на PS4 различается**

Здесь нам уже нужен двухвыборочный двухсторонний критерий.

In [6]:
result2 = st.ttest_ind(df_crit.loc[df_crit['Platform'] =='PC'].Critic_Score,\
                      df_crit.loc[df_crit['Platform'] =='PS4'].Critic_Score, equal_var=False)
print(result2)
if (result2.pvalue < alpha) & (result2.statistic > 0):
    print('Отвергаем нулевую гипотезу, критики выше оценивают игры на платформе PC')
elif (result2.pvalue < alpha) & (result2.statistic < 0):
    print('Отвергаем нулевую гипотезу, критики выше оценивают игры на платформе PS4')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, критики выше оценивают игры на платформе PS4


**3) Критикам больше нравятся стрелялки или стратегии?** 

**H0: средняя оценка для Shooter и Strategy одинаковая**    
**H1: средняя оценка для Shooter и Strategy различается**

In [7]:
result3 = st.ttest_ind(df_crit.loc[df_crit['Genre'] =='Shooter'].Critic_Score,\
                      df_crit.loc[df_crit['Genre'] =='Strategy'].Critic_Score, equal_var=False)
print(result3)
if (result3.pvalue < alpha) & (result3.statistic > 0):
    print('Отвергаем нулевую гипотезу, критики выше оценивают игры типа Стрелялки')
elif (result3.pvalue < alpha) & (result3.statistic < 0):
    print('Отвергаем нулевую гипотезу, критики выше оценивают Стратегии ')
else:
    print('Не отвергаем нулевую гипотезу')
    

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу


## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. 

In [8]:
df_spam = pd.read_csv('spam.csv')
df_spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [9]:
def message_transform(row):
    row = row.lower() # Приводим текст к нижнему регистру
    row = re.sub('[\W_]+',' ', row) #  Удаляем мусорные символы
    row = row.split()
    row = [word for word in row if not word in set(stopwords.words('english'))] # Удаляем стоп-слова
    row = [wordnet_lemmatizer.lemmatize(word) for word in row] # приводим все слова к нормальной форме
    row = ' '.join(map(str, row)) # возвращаем в строку
    return row

In [10]:
df_spam.Message = df_spam.Message.apply(message_transform)
df_spam.head()

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


**1) Приводим весь текст к нижнему регистру;**

In [11]:
df_spam.Message = df_spam.Message.str.lower()

**2) Удаляем мусорные символы;**

In [12]:
df_spam.Message = df_spam.Message.replace('[\W_]+', ' ', regex=True)

**3) Удаляем стоп-слова**

In [13]:
df_spam.Message = df_spam.Message.str.split()
stopwords_set = set(stopwords.words('english'))
df_spam.Message = df_spam.Message.apply(lambda x: [word for word in x if word not in stopwords_set])

**4) Приводим все слова к нормальной форме**

In [14]:
df_spam.Message = df_spam.Message.apply(lambda x: [wordnet_lemmatizer.lemmatize(word) for word in x])
# возвращаем в строку
df_spam.Message = df_spam.Message.str.join(" ")
df_spam

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though
...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,ü b going esplanade fr home
5569,ham,pity mood suggestion
5570,ham,guy bitching acted like interested buying some...


**5) Преобразуем все сообщения в вектора TF-IDF**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_spam.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  


**6) Разделим данные на тестовые и тренировочные** в соотношении 30/70, укажем `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_spam['Category'], test_size = 0.30, random_state = 42)

**7) Постройим модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)**, укажем `random_state=42`, **оценим ее точность на тестовых данных**;  


In [18]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, lr_model.predict(X_test))

0.958732057416268

**8) Опишем результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);** 


In [20]:
from sklearn.metrics import confusion_matrix

In [21]:
confusion_matrix(y_test, lr_model.predict(X_test))

array([[1445,    3],
       [  66,  158]], dtype=int64)

**9) Построим датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).**

In [23]:
predict = pd.DataFrame(lr_model.predict(tfidf_matrix))
predict.columns = ['Prediction']
predict

,Prediction
0,ham
1,ham
2,spam
3,ham
4,ham
...,...
5567,spam
5568,ham
5569,ham
5570,ham


In [25]:
confusion_matrix(df_spam['Category'], predict)

array([[4816,    9],
       [ 197,  550]], dtype=int64)

In [29]:
result = predict.join(df_spam, how = 'left')
result = result.loc[result['Prediction'] != result['Category']]
result

,Prediction,Category,Message
5,ham,spam,freemsg hey darling 3 week word back like fun ...
15,ham,spam,xxxmobilemovieclub use credit click wap link n...
19,ham,spam,england v macedonia dont miss goal team news t...
54,ham,spam,sm ac sptv new jersey devil detroit red wing p...
56,ham,spam,congrats 1 year special cinema pa 2 call 09061...
...,...,...,...
5449,ham,spam,latest news police station toilet stolen cop n...
5456,ham,spam,sparkling shopping break 45 per person call 01...
5466,ham,spam,http tm widelive com index wml id 820554ad0a17...
5537,ham,spam,want explicit sex 30 sec ring 02073162414 cost...


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
